# Data Wrangling with Spark SQL Quiz

This quiz uses the same dataset and most of the same questions from the earlier "Quiz - Data Wrangling with Data Frames Jupyter Notebook." For this quiz, however, use Spark SQL instead of Spark Data Frames.

In [2]:
# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) create a view to use with your SQL queries
# 5) write code to answer the quiz questions 

from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc

import datetime

In [3]:
spark = SparkSession \
    .builder \
    .appName('Spark Exercises 2') \
    .getOrCreate()


22/04/18 11:30:21 WARN Utils: Your hostname, Erichs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.15.5 instead (on interface en0)
22/04/18 11:30:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/18 11:30:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/04/18 11:30:22 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
path = "data/sparkify_log_small.json"
user_log = spark.read.json(path)

In [6]:
user_log.take(1)

[Row(artist='Showaddywaddy', auth='Logged In', firstName='Kenneth', gender='M', itemInSession=112, lastName='Matthews', length=232.93342, level='paid', location='Charlotte-Concord-Gastonia, NC-SC', method='PUT', page='NextSong', registration=1509380319284, sessionId=5132, song='Christmas Tears Will Fall', status=200, ts=1513720872284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='1046')]

In [9]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [5]:
user_log.createOrReplaceTempView("user_log_table")

# Question 1

Which page did user id ""(empty string) NOT visit?

In [12]:
# TODO: write your code to answer question 1

spark.sql(
    '''
    select
        a.page
    from (
        select distinct
            page
        from user_log_table
        ) a
    left join (
                select distinct
                    page
                from user_log_table
                where
                    userId = ""
            ) b
    on  a.page = b.page
    where
        b.page is null
    '''
).show()



+----------------+
|            page|
+----------------+
|Submit Downgrade|
|       Downgrade|
|          Logout|
|   Save Settings|
|        Settings|
|        NextSong|
|         Upgrade|
|           Error|
|  Submit Upgrade|
+----------------+



# Question 2 - Reflect

Why might you prefer to use SQL over data frames? Why might you prefer data frames over SQL?

I might prefer to use SQL because I'm more skilled and used to SQL syntaxes and I could easily share the code with other analysts with more familiarity with SQL.

I think I would might prefer data frames over SQL because the control I would have over data, like split my queries in more steps.

# Question 3

How many female users do we have in the data set?

In [14]:
# TODO: write your code to answer question 3

spark.sql('''
    select
        gender
        ,count(distinct userId) as qty
    from user_log_table
    where
        gender = "F"
    group by
        gender
''').show()

+------+---+
|gender|qty|
+------+---+
|     F|462|
+------+---+



# Question 4

How many songs were played from the most played artist?

In [18]:
# TODO: write your code to answer question 4

spark.sql('''
    select
        artist
        ,count(artist) as qty
    from user_log_table
    group by
        artist
    order by
        qty desc
    limit 1 
''').show()

+--------+---+
|  artist|qty|
+--------+---+
|Coldplay| 83|
+--------+---+



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.

In [24]:
# TODO: write your code to answer question 5

home_totals = spark.sql('''
    with is_home as (
        select
            userId
            ,page
            ,ts
            ,case
                when page = 'Home'
                    then 1
                    else 0
            end as ishome 
        from user_log_table 
        where
            (page = 'NextSong') or (page = 'Home')
    )

    select
        *
        ,sum(ishome) over(partition by userId order by ts desc rows between unbounded preceding and current row) as window_qty
    from is_home
            ''')

home_totals.createOrReplaceTempView('tmp_window_qty')

spark.sql('''
    select
        avg(total_qty)
    from (
        select
            count(*) as total_qty
        from tmp_window_qty
        group by
            userID
            ,window_qty
            ,page
        having page = 'NextSong') as count_results'''
    ).show()

+-----------------+
|   avg(total_qty)|
+-----------------+
|6.898347107438017|
+-----------------+

